<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [25]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [26]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [27]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [28]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [29]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_0

,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0),y1(0),y2(0),y3(0)
0,0.629963,0.906652,-0.896224,2.439534,37.362450,1.220939,0.215251,0.969238,1.220939
1,0.972471,-0.977731,-0.923388,0.989036,-53.776059,0.607146,0.980603,0.755331,0.607146
2,-0.532114,-0.357306,-0.882230,3.265018,91.048491,16.079237,-0.198885,0.602331,16.079237
3,0.427549,-0.975179,-0.290523,7.441486,-46.535179,2.461357,0.038043,0.793080,2.461357
4,0.923399,-0.497913,-0.798611,-1.791358,-12.102322,3.960247,-0.701981,0.854533,3.960247
...,...,...,...,...,...,...,...,...,...
95,-0.699550,0.808315,-0.954870,1.643045,-29.725639,0.527795,-0.396085,0.709668,0.527795
96,-0.368282,0.177397,0.391217,1.930776,93.524243,1.015312,-0.274831,0.921762,1.015312
97,-0.035213,-0.878752,-0.901050,0.176637,54.260314,17.633814,-0.039906,0.968312,17.633814
98,-0.180425,-0.795939,0.310159,0.576394,-33.311378,9.830599,-0.149035,0.628310,9.830599


In [30]:
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_1

,x1(1),x2(1),x3(1),v1(1),v2(1),v3(1),y1(1),y2(1),y3(1)
0,0.824260,0.407525,-0.541174,6.222005,-65.924638,6.081773,0.188404,0.447273,6.081773
1,0.914287,-0.505202,0.969483,-4.988396,86.007740,2.346545,-0.246565,0.844924,2.346545
2,0.904251,-0.981112,-0.172611,17.356248,27.833685,-1.070082,0.050017,0.936611,-1.070082
3,-0.052100,-0.652748,-0.748350,7.993135,40.166808,15.601511,-0.001842,0.325962,15.601511
4,0.152818,0.373290,-0.784454,15.678567,-86.053874,3.063150,0.006236,0.279932,3.063150
...,...,...,...,...,...,...,...,...,...
95,0.344013,0.571647,-0.369842,2.960178,-28.087551,3.622213,0.090153,0.679948,3.622213
96,-0.791876,0.788693,0.965318,0.861483,-28.296249,-1.031290,-0.921055,0.926262,-1.031290
97,0.962042,0.465372,-0.353766,0.048153,60.789647,4.780753,28.725622,0.998831,4.780753
98,-0.968488,-0.198646,-0.975127,-0.249055,75.622914,1.131150,8.586308,0.951264,1.131150


In [31]:
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_2

,x1(2),x2(2),x3(2),v1(2),v2(2),v3(2),y1(2),y2(2),y3(2)
0,-0.956073,-0.979783,0.885483,0.400850,-70.303027,11.467611,-2.356077,0.943740,11.467611
1,-0.983956,-0.017726,-0.960387,-1.607122,38.975461,1.497380,4.561527,0.998652,1.497380
2,0.086539,-0.292060,-0.983466,0.995789,-94.294198,0.724092,0.047306,0.813914,0.724092
3,0.893805,0.935154,0.122626,22.554245,-1.934767,9.908962,0.038743,0.682266,9.908962
4,0.758174,0.930506,-0.383282,13.389340,-42.214550,0.365773,0.051113,0.598056,0.365773
...,...,...,...,...,...,...,...,...,...
95,-0.650061,-0.855764,0.389120,3.090788,-74.242812,9.389648,-0.183310,0.187412,9.389648
96,-0.285191,0.959322,0.752262,-8.653540,-99.691995,1.007030,0.017969,0.919374,1.007030
97,0.815498,-0.635443,-0.619337,13.947522,58.344792,1.348072,0.066237,0.987572,1.348072
98,0.961146,0.972652,-0.536576,31.389576,75.601436,5.554355,0.030438,0.816087,5.554355


##1. Линейная форма функции пересчёта

###Для функции $x_1[t+1]$

In [32]:
#x3, v1, v2
vector_ones = [1] * elem
x1_linearFunc = model_0[['x3(0)', 'v1(0)', 'v2(0)']]
x1_linearFunc['x1(1)'] = model_1['x1(1)']
x1_linearFunc.insert(0, "optional_col", vector_ones)

A1_linear = np.array(x1_linearFunc)
#A1_psev = BenIsrael(A1_linear[:, 0:(len(A1_linear[0]) - 1)])
X1_linear = solvingSLAE(A1_linear[:, 0:(len(A1_linear[0]) - 1)], np.matrix(A1_linear[:, len(A1_linear[0]) - 1]).T)
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c13', 'b11', 'b12']))

           X1
a1   0.043921
c13 -0.478005
b11  0.000096
b12  0.000414


<ipython-input-32-e7a1fb8a3e1a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['x1(1)'] = model_1['x1(1)']


In [33]:
X1_estimate_linear = []
X1_variance_linear = []
X1_error_linear = []
for i in range(len(x1_linearFunc)):
  X1_estimate_linear.append(X1_linear[0,0] + X1_linear[1,0] * np.array(x1_linearFunc)[i,1]
                            + X1_linear[2,0] * np.array(x1_linearFunc)[i,2] + X1_linear[3,0] * np.array(x1_linearFunc)[i,3])

  X1_variance_linear.append((1.0 / len(x1_linearFunc)) * (np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1] - X1_estimate_linear[i])**2)

  X1_error_linear.append((1.0 / len(x1_linearFunc)) * abs((np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1] - X1_estimate_linear[i])
                            / np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1]) * 100)

x1_linearFunc['X1*'] = X1_estimate_linear
x1_linearFunc['(X1 - X1*)^2'] = X1_variance_linear
x1_linearFunc['|(X1 - X1*)| / |X1|, %'] = X1_error_linear
x1_linearFunc

<ipython-input-33-65ca3a007bdb>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['X1*'] = X1_estimate_linear
<ipython-input-33-65ca3a007bdb>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['(X1 - X1*)^2'] = X1_variance_linear
<ipython-input-33-65ca3a007bdb>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,x3(0),v1(0),v2(0),x1(1),X1*,(X1 - X1*)^2,"|(X1 - X1*)| / |X1|, %"
0,1,-0.896224,2.439534,37.362450,0.824260,0.488041,0.001130,0.407905
1,1,-0.923388,0.989036,-53.776059,0.914287,0.463114,0.002036,0.493470
2,1,-0.882230,3.265018,91.048491,0.904251,0.503681,0.001605,0.442985
3,1,-0.290523,7.441486,-46.535179,-0.052100,0.164224,0.000468,4.152054
4,1,-0.798611,-1.791358,-12.102322,0.152818,0.420473,0.000716,1.751459
...,...,...,...,...,...,...,...,...
95,1,-0.954870,1.643045,-29.725639,0.344013,0.488192,0.000208,0.419110
96,1,0.391217,1.930776,93.524243,-0.791876,-0.104135,0.004730,0.868496
97,1,-0.901050,0.176637,54.260314,0.962042,0.497133,0.002161,0.483253
98,1,0.310159,0.576394,-33.311378,-0.968488,-0.118086,0.007232,0.878071


###Для функции $x_2[t+1]$

In [39]:
#x2, x3, v1
x2_linearFunc = model_0[['x2(0)', 'x3(0)', 'v1(0)']]
x2_linearFunc['x2(1)'] = model_1['x2(1)']
x2_linearFunc.insert(0, "optional_col", vector_ones)

A2_linear = np.array(x2_linearFunc)
#A2_psev = BenIsrael(A2_linear[:, 0:(len(A2_linear[0]) - 1)])
X2_linear = solvingSLAE(A2_linear[:, 0:(len(A2_linear[0]) - 1)], np.matrix(A2_linear[:, len(A2_linear[0]) - 1]).T)
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'c22', 'c23', 'b21']))

           X2
a2  -0.013959
c22  0.215477
c23  0.123312
b21 -0.000002


<ipython-input-39-39802f8807a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linearFunc['x2(1)'] = model_1['x2(1)']


In [41]:
X2_estimate_linear = []
X2_variance_linear = []
X2_error_linear = []
for i in range(len(x2_linearFunc)):
  X2_estimate_linear.append(X2_linear[0,0] + X2_linear[1,0] * np.array(x2_linearFunc)[i,1]
                            + X2_linear[2,0] * np.array(x2_linearFunc)[i,2] + X2_linear[3,0] * np.array(x2_linearFunc)[i,3])

  X2_variance_linear.append((1.0 / len(x2_linearFunc)) * (np.array(x2_linearFunc)[i, len(np.array(x2_linearFunc)[0]) - 1] - X2_estimate_linear[i])**2)

  X2_error_linear.append((1.0 / len(x2_linearFunc)) * abs((np.array(x2_linearFunc)[i, len(np.array(x2_linearFunc)[0]) - 1] - X2_estimate_linear[i])
                            / np.array(x2_linearFunc)[i, len(np.array(x2_linearFunc)[0]) - 1]) * 100)

x2_linearFunc['X2*'] = X2_estimate_linear
x2_linearFunc['(X2 - X2*)^2'] = X2_variance_linear
x2_linearFunc['|(X2 - X2*)| / |X2|, %'] = X2_error_linear
x2_linearFunc

,optional_col,x2(0),x3(0),v1(0),x2(1),X1*,(X1 - X1*)^2,"|(X1 - X1*)| / |X1|, %",X2*,(X2 - X2*)^2,"|(X2 - X2*)| / |X2|, %"
0,1,0.906652,-0.896224,2.439534,0.407525,0.488041,0.001130,0.407905,0.070883,0.001136,0.826227
1,1,-0.977731,-0.923388,0.989036,-0.505202,0.463114,0.002036,0.493470,-0.338504,0.006922,1.685966
2,1,-0.357306,-0.882230,3.265018,-0.981112,0.503681,0.001605,0.442985,-0.199747,0.004131,1.450910
3,1,-0.975179,-0.290523,7.441486,-0.652748,0.164224,0.000468,4.152054,-0.259930,0.194656,1.062603
4,1,-0.497913,-0.798611,-1.791358,0.373290,0.420473,0.000716,1.751459,-0.219721,0.038856,1.125450
...,...,...,...,...,...,...,...,...,...,...,...
95,1,0.808315,-0.954870,1.643045,0.571647,0.488192,0.000208,0.419110,0.042464,0.001419,0.898681
96,1,0.177397,0.391217,1.930776,0.788693,-0.104135,0.004730,0.868496,0.072504,0.006336,0.916518
97,1,-0.878752,-0.901050,0.176637,0.465372,0.497133,0.002161,0.483253,-0.314420,0.006363,1.650632
98,1,-0.795939,0.310159,0.576394,-0.198646,-0.118086,0.007232,0.878071,-0.147220,0.010512,1.167663


###Для функции $x_3[t+1]$

In [42]:
#x3, v1, v3
x3_linearFunc = model_0[['x3(0)', 'v1(0)', 'v3(0)']]
x3_linearFunc['x3(1)'] = model_1['x3(1)']
x3_linearFunc.insert(0, "optional_col", vector_ones)

A3_linear = np.array(x3_linearFunc)
#A3_psev = BenIsrael(A3_linear[:, 0:(len(A3_linear[0]) - 1)])
X3_linear = solvingSLAE(A3_linear[:, 0:(len(A3_linear[0]) - 1)], np.matrix(A3_linear[:, len(A3_linear[0]) - 1]).T)
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c33', 'b31', 'b33']))

           X3
a3  -0.113454
c33 -0.033824
b31  0.000442
b33 -0.015562


<ipython-input-42-f740405435c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linearFunc['x3(1)'] = model_1['x3(1)']


In [44]:
X3_estimate_linear = []
X3_variance_linear = []
X3_error_linear = []
for i in range(len(x3_linearFunc)):
  X3_estimate_linear.append(X3_linear[0,0] + X3_linear[1,0] * np.array(x3_linearFunc)[i,1]
                            + X3_linear[2,0] * np.array(x3_linearFunc)[i,2] + X3_linear[3,0] * np.array(x3_linearFunc)[i,3])

  X3_variance_linear.append((1.0 / len(x3_linearFunc)) * (np.array(x3_linearFunc)[i, len(np.array(x3_linearFunc)[0]) - 1] - X3_estimate_linear[i])**2)

  X3_error_linear.append((1.0 / len(x3_linearFunc)) * abs((np.array(x3_linearFunc)[i, len(np.array(x3_linearFunc)[0]) - 1] - X3_estimate_linear[i])
                            / np.array(x3_linearFunc)[i, len(np.array(x3_linearFunc)[0]) - 1]) * 100)

x3_linearFunc['X3*'] = X3_estimate_linear
x3_linearFunc['(X3 - X3*)^2'] = X3_variance_linear
x3_linearFunc['|(X3 - X3*)| / |X3|, %'] = X3_error_linear
x3_linearFunc

<ipython-input-44-0f73eb653e66>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linearFunc['X3*'] = X3_estimate_linear
<ipython-input-44-0f73eb653e66>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linearFunc['(X3 - X3*)^2'] = X3_variance_linear
<ipython-input-44-0f73eb653e66>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,x3(0),v1(0),v3(0),x3(1),X3*,(X3 - X3*)^2,"|(X3 - X3*)| / |X3|, %"
0,1,-0.896224,2.439534,1.220939,-0.541174,-0.101062,0.008360,1.124269
1,1,-0.923388,0.989036,0.607146,0.969483,-0.091233,0.014050,1.083386
2,1,-0.882230,3.265018,16.079237,-0.172611,-0.332400,0.015828,1.359074
3,1,-0.290523,7.441486,2.461357,-0.748350,-0.138641,0.009089,1.170167
4,1,-0.798611,-1.791358,3.960247,-0.784454,-0.148865,0.009199,1.183731
...,...,...,...,...,...,...,...,...
95,1,-0.954870,1.643045,0.527795,-0.369842,-0.088644,0.007207,1.116587
96,1,0.391217,1.930776,1.015312,0.965318,-0.141634,0.016599,1.123512
97,1,-0.901050,0.176637,17.633814,-0.353766,-0.357322,0.001350,36.556031
98,1,0.310159,0.576394,9.830599,-0.975127,-0.276677,0.009859,1.386277


###Для функции $y_1[t]$

In [45]:
#x1, x2, v1
y1_linearFunc = model_0[['x1(0)', 'x2(0)', 'v1(0)']]
y1_linearFunc['y1(0)'] = model_0['y1(0)']
y1_linearFunc.insert(0, "optional_col", vector_ones)

A1y_linear = np.array(y1_linearFunc)
#A1y_psev = BenIsrael(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)])
Y1_linear = solvingSLAE(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)], np.matrix(A1y_linear[:, len(A1y_linear[0]) - 1]).T)
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'c12', 'b11']))

           Y1
a1   0.682127
c11 -1.808600
c12 -0.459276
b11 -0.001040


<ipython-input-45-2e41166f56a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linearFunc['y1(0)'] = model_0['y1(0)']


In [46]:
Y1_estimate_linear = []
Y1_variance_linear = []
Y1_error_linear = []
for i in range(len(x3_linearFunc)):
  Y1_estimate_linear.append(Y1_linear[0,0] + Y1_linear[1,0] * np.array(y1_linearFunc)[i,1]
                            + Y1_linear[2,0] * np.array(y1_linearFunc)[i,2] + Y1_linear[3,0] * np.array(y1_linearFunc)[i,3])

  Y1_variance_linear.append((1.0 / len(y1_linearFunc)) * (np.array(y1_linearFunc)[i, len(np.array(y1_linearFunc)[0]) - 1] - Y1_estimate_linear[i])**2)

  Y1_error_linear.append((1.0 / len(y1_linearFunc)) * abs((np.array(y1_linearFunc)[i, len(np.array(y1_linearFunc)[0]) - 1] - Y1_estimate_linear[i])
                            / np.array(y1_linearFunc)[i, len(np.array(y1_linearFunc)[0]) - 1]) * 100)

y1_linearFunc['Y1*'] = Y1_estimate_linear
y1_linearFunc['(Y1 - Y1*)^2'] = Y1_variance_linear
y1_linearFunc['|(Y1 - Y1*)| / |Y1|, %'] = Y1_error_linear
y1_linearFunc

<ipython-input-46-20857d005a00>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linearFunc['Y1*'] = Y1_estimate_linear
<ipython-input-46-20857d005a00>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linearFunc['(Y1 - Y1*)^2'] = Y1_variance_linear
<ipython-input-46-20857d005a00>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,x1(0),x2(0),v1(0),y1(0),Y1*,(Y1 - Y1*)^2,"|(Y1 - Y1*)| / |Y1|, %"
0,1,0.629963,0.906652,2.439534,0.215251,-0.876164,0.011912,5.070428
1,1,0.972471,-0.977731,0.989036,0.980603,-0.628663,0.025897,1.641099
2,1,-0.532114,-0.357306,3.265018,-0.198885,1.805216,0.040164,10.076692
3,1,0.427549,-0.975179,7.441486,0.038043,0.349002,0.000967,8.173856
4,1,0.923399,-0.497913,-1.791358,-0.701981,-0.757391,0.000031,0.078934
...,...,...,...,...,...,...,...,...
95,1,-0.699550,0.808315,1.643045,-0.396085,1.574385,0.038828,4.974867
96,1,-0.368282,0.177397,1.930776,-0.274831,1.264721,0.023702,5.601813
97,1,-0.035213,-0.878752,0.176637,-0.039906,1.149220,0.014140,29.797828
98,1,-0.180425,-0.795939,0.576394,-0.149035,1.373401,0.023178,10.215317


###Для функции $y_2[t]$

In [48]:
#v3
y2_linearFunc = model_0[['v3(0)']]
y2_linearFunc['y2(0)'] = model_0['y2(0)']
y2_linearFunc.insert(0, "optional_col", vector_ones)

A2y_linear = np.array(y2_linearFunc)
#A2y_psev = BenIsrael(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)])
Y2_linear = solvingSLAE(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)], np.matrix(A2y_linear[:, len(A2y_linear[0]) - 1]).T)
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b23']))

           Y2
a2   0.770338
b23  0.000612


<ipython-input-48-a0e07d78ce80>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linearFunc['y2(0)'] = model_0['y2(0)']


In [50]:
Y2_estimate_linear = []
Y2_variance_linear = []
Y2_error_linear = []
for i in range(len(y2_linearFunc)):
  Y2_estimate_linear.append(Y2_linear[0,0] + Y2_linear[1,0] * np.array(y2_linearFunc)[i,1])

  Y2_variance_linear.append((1.0 / len(y2_linearFunc)) * (np.array(y2_linearFunc)[i, len(np.array(y2_linearFunc)[0]) - 1] - Y2_estimate_linear[i])**2)

  Y2_error_linear.append((1.0 / len(y2_linearFunc)) * abs((np.array(y2_linearFunc)[i, len(np.array(y2_linearFunc)[0]) - 1] - Y2_estimate_linear[i])
                            / np.array(y2_linearFunc)[i, len(np.array(y2_linearFunc)[0]) - 1]) * 100)

y2_linearFunc['Y2*'] = Y2_estimate_linear
y2_linearFunc['(Y2 - Y2*)^2'] = Y2_variance_linear
y2_linearFunc['|(Y2 - Y2*)| / |Y2|, %'] = Y2_error_linear
y2_linearFunc

<ipython-input-50-7cd0bf4dc686>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linearFunc['Y2*'] = Y2_estimate_linear
<ipython-input-50-7cd0bf4dc686>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linearFunc['(Y2 - Y2*)^2'] = Y2_variance_linear
<ipython-input-50-7cd0bf4dc686>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,v3(0),y2(0),Y2*,(Y2 - Y2*)^2,"|(Y2 - Y2*)| / |Y2|, %"
0,1,1.220939,0.969238,0.771085,0.000393,0.204442
1,1,0.607146,0.755331,0.770709,0.000002,0.020359
2,1,16.079237,0.602331,0.780177,0.000316,0.295264
3,1,2.461357,0.793080,0.771844,0.000005,0.026777
4,1,3.960247,0.854533,0.772761,0.000067,0.095692
...,...,...,...,...,...,...
95,1,0.527795,0.709668,0.770661,0.000037,0.085945
96,1,1.015312,0.921762,0.770959,0.000227,0.163603
97,1,17.633814,0.968312,0.781129,0.000350,0.193309
98,1,9.830599,0.628310,0.776353,0.000219,0.235621


###Для функции $y_3[t]$

In [51]:
#v1, v3
y3_linearFunc = model_0[['v1(0)', 'v3(0)']]
y3_linearFunc['y3(0)'] = model_0['y3(0)']
y3_linearFunc.insert(0, "optional_col", vector_ones)

A3y_linear = np.array(y3_linearFunc)
#A3y_psev = BenIsrael(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)])
Y3_linear = solvingSLAE(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)], np.matrix(A3y_linear[:, len(A3y_linear[0]) - 1]).T)
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'b31', 'b33']))

               Y3
a3   1.617069e-08
b31 -1.177733e-11
b33  1.000000e+00


<ipython-input-51-b0b21a59e704>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linearFunc['y3(0)'] = model_0['y3(0)']


In [53]:
Y3_estimate_linear = []
Y3_variance_linear = []
Y3_error_linear = []
for i in range(len(y3_linearFunc)):
  Y3_estimate_linear.append(Y3_linear[0,0] + Y3_linear[1,0] * np.array(y3_linearFunc)[i,1]
                            + Y3_linear[2,0] * np.array(y3_linearFunc)[i,2])

  Y3_variance_linear.append((1.0 / len(y3_linearFunc)) * (np.array(y3_linearFunc)[i, len(np.array(y3_linearFunc)[0]) - 1] - Y3_estimate_linear[i])**2)

  Y3_error_linear.append((1.0 / len(y3_linearFunc)) * abs((np.array(y3_linearFunc)[i, len(np.array(y3_linearFunc)[0]) - 1] - Y3_estimate_linear[i])
                            / np.array(y3_linearFunc)[i, len(np.array(y3_linearFunc)[0]) - 1]) * 100)

y3_linearFunc['Y3*'] = Y3_estimate_linear
y3_linearFunc['(Y3 - Y3*)^2'] = Y3_variance_linear
y3_linearFunc['|(Y3 - Y3*)| / |Y3|, %'] = Y3_error_linear
y3_linearFunc

<ipython-input-53-e73931a3ed05>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linearFunc['Y3*'] = Y3_estimate_linear
<ipython-input-53-e73931a3ed05>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linearFunc['(Y3 - Y3*)^2'] = Y3_variance_linear
<ipython-input-53-e73931a3ed05>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,v1(0),v3(0),y3(0),Y3*,(Y3 - Y3*)^2,"|(Y3 - Y3*)| / |Y3|, %"
0,1,2.439534,1.220939,1.220939,1.220939,1.966428e-18,1.148537e-08
1,1,0.989036,0.607146,0.607146,0.607146,2.281700e-18,2.487919e-08
2,1,3.265018,16.079237,16.079237,16.079236,1.386342e-18,7.322674e-10
3,1,7.441486,2.461357,2.461357,2.461357,1.395045e-18,4.798655e-09
4,1,-1.791358,3.960247,3.960247,3.960247,8.683488e-19,2.353016e-09
...,...,...,...,...,...,...,...
95,1,1.643045,0.527795,0.527795,0.527795,2.321148e-18,2.886599e-08
96,1,1.930776,1.015312,1.015312,1.015312,2.069515e-18,1.416886e-08
97,1,0.176637,17.633814,17.633814,17.633814,2.083980e-18,8.186543e-10
98,1,0.576394,9.830599,9.830599,9.830599,8.059221e-21,9.132014e-11
